# Step 3. Predict docking poses

## 3.1 Set up environement ##

In [ ]:
!pip install pandas numpy matplotlib loguru py3dmol rdkit ipywidgets

In [ ]:
import time
from loguru import logger
import os, shutil
from google.colab import files
import zipfile
import requests
from google.colab import userdata
import json
import py3Dmol
from rdkit import Chem
import ipywidgets as widgets
from IPython.display import display
import glob
import random

In [ ]:
def prepare_output_directory(output):
    """
    Prepare the output directory
    output: str, the output directory
    return: None
    """
    # overwrite the output directory
    # delete the output directory if it exists
    if os.path.exists(output):
        shutil.rmtree(output)
    os.makedirs(output)

## 3.2 Set up directories

Upload the structure file for the predicted target protein

In [ ]:
protein_dir = "/content/output/esmfold_result"
prepare_output_directory(protein_dir)

In [ ]:
# choose to upload "predicted_protein.pdb" which was downloaded at the end of Step 1.
uploaded = files.upload()

In [ ]:
# Move the uploaded file to the target folder
for filename in uploaded.keys():
    !mv "{filename}" "{protein_dir}/{filename}"

In [ ]:
# file path of the predicted target protein
protein_file_path = os.path.join(protein_dir, list(uploaded.keys())[0])
print(protein_file_path)

Upload ligand files

In [ ]:
ligand_dir = "/content/output/molmim_result"
prepare_output_directory(ligand_dir)

In [ ]:
# choose to upload clean_mol.zip which wasdownloaded at the end of Step 2
uploaded = files.upload()

In [ ]:
zip_filename = list(uploaded.keys())[0]
print(zip_filename)

In [ ]:
with zipfile.ZipFile(zip_filename, 'r') as zip_ref:
    zip_ref.extractall(ligand_dir)

In [ ]:
!ls {ligand_dir}

Set up output directory for docking

In [ ]:
docking_dir = "/content/output/diffdock_result"
prepare_output_directory(docking_dir)

## 3.3 Predict

Load ligands

In [ ]:
# Load all SDF files from the specified directory
sdf_files = [f for f in os.listdir(ligand_dir) if f.endswith(".sdf")]

# Sort ligand files based on the numeric part in the filename (molecule_0, molecule_1, molecule_2 ....)
sdf_files.sort(key=lambda x: int(x.split("_")[1].split(".")[0]))

# Add a prefix directory path to each file in sdf_files
sdf_files = [os.path.join(ligand_dir, f) for f in sdf_files]

# Get name of the sdf files
ligand_names = [os.path.basename(f).split(".")[0] for f in sdf_files]

print(sdf_files)
print(ligand_names)

In [ ]:
# For demo purpose, we'll only use the first two ligands
sdf_files = sdf_files[:2]
ligand_names = ligand_names[:2]
print(sdf_files)
print(ligand_names)

Set up API_Key

In [ ]:
API_KEY = userdata.get('API_KEY')
print(API_KEY)
header_auth = f"Bearer {API_KEY}"
print(header_auth)

 Upload the target protein to AWS S3 Storage

In [ ]:
# get asset-uploading URL & upload the asset to AWS S3 storag
def _upload_asset(input):
    assets_url = "https://api.nvcf.nvidia.com/v2/nvcf/assets"

    headers = {
        "Authorization": header_auth,
        "Content-Type": "application/json",
        "accept": "application/json",
    }

    s3_headers = {
        "x-amz-meta-nvcf-asset-description": "diffdock-file",
        "content-type": "text/plain",
    }

    payload = {
        "contentType": "text/plain",
        "description": "diffdock-file"
    }

    response = requests.post(
        assets_url, headers=headers, json=payload, timeout=30
    )

    response.raise_for_status()

    asset_url = response.json()["uploadUrl"]
    asset_id = response.json()["assetId"]

    response = requests.put(
        asset_url,
        data=input,
        headers=s3_headers,
        timeout=300,
    )

    response.raise_for_status()
    return asset_id

In [ ]:
invoke_url = "https://health.api.nvidia.com/v1/biology/mit/diffdock"

In [ ]:
# get asset-uploading URL & upload assets for target protein
with open(protein_file_path, "r") as file:
    pdb_content = file.read()
    protein_id = _upload_asset(pdb_content)
print(protein_id)

Main loop to iterate over all ligands to generate docking poses

In [ ]:
# Iterate over ligands' SDF files
for ligand_file_path, ligand_name in zip(sdf_files, ligand_names):
    print(f"************ {ligand_name} ****************")
    # get asset-uploading URL & upload assets for ligand
    with open(ligand_file_path, "r") as file:
        sdf_content = file.read()
        ligand_id = _upload_asset(sdf_content)
    print(f"ligand_id: {ligand_id}")

    # DiffDock inference
    headers = {
        "Content-Type": "application/json",
        "NVCF-INPUT-ASSET-REFERENCES": ",".join([protein_id, ligand_id]),
        "Authorization": header_auth
    }

    payload = {
        "ligand": ligand_id,
        "ligand_file_type": "sdf",
        "protein": protein_id,
        "num_poses": 3,
        "time_divisions": 20,
        "steps": 18,
        "save_trajectory": False,
        "is_staged": True
    }

    start = time.time()
    response = requests.post(invoke_url, headers=headers, json=payload)
    end = time.time()
    logger.debug(f"{ligand_name} took {end - start:.2f} seconds")

    response.raise_for_status()

    result = response.json()

    # save result to output.json
    docking_ligand_dir = os.path.join(docking_dir, ligand_name)
    prepare_output_directory(docking_ligand_dir)
    with open(f"{docking_ligand_dir}/output.json", "w") as f:
        json.dump(result, f)

    # save ligand positions
    for i, ligand_geometry in enumerate(result["ligand_positions"]):
        with open("{}/pose_{}_confidence_{:.2f}.sdf".format(docking_ligand_dir, i, result["position_confidence"][i]), "w") as f:
            f.write(ligand_geometry)

## 3.4 Visualize the docking poses

In [ ]:
# assume we select molecule_0
ligand_name = "molecule_0"

docking_ligand_dir = os.path.join(docking_dir, ligand_name)
# take a look at the JSON output file
with open(f"{docking_ligand_dir}/output.json", "r") as f:
    result = json.load(f)
result.keys()

- `trajectory`: diffusion trajectory (empty unless `save_trajectory` is set to `True`)
- `ligand_positions`: a list of docking poses
- `ligand_scores`: a list of confidence scores for each docking pose
- `protein`: input protein
- `ligand`: input ligand

Confidence score the logits of the probability that the docked pose has a RMSD < 2A compared to ground truth. Interpretation of confidence score (c) is based on the guideline provided by [github authors](https://github.com/gcorso/DiffDock?tab=readme-ov-file#faq--).
```
c > 0 : high confidence
-1.5 < c < 0: moderate confidence
c < -1.5: low confidence
```

Visusalize docking poses and [confidence score](https://github.com/gcorso/DiffDock#faq--)

In [ ]:
# define a function for color definitions for visualization
def ansi_color(text, color):
    """Color text for console output"""
    colors = {
        "red": "\033[31m",
        "green": "\033[32m",
        "yellow": "\033[33m",
        "blue": "\033[34m",
        "magenta": "\033[35m",
        "cyan": "\033[36m",
        "white": "\033[37m",
        "reset": "\033[0m"
    }
    return f"{colors[color]}{text}{colors['reset']}"

In [ ]:
# load docking poses from the output SDF files extracted from the output.json 'positions' field
def load_poses_from_sdf(directory):
    sdf_files = glob.glob(f"{directory}/*.sdf")
    poses = []

    for sdf_file in sdf_files:
        supplier = Chem.SDMolSupplier(sdf_file)
        for mol in supplier:
            if mol is not None:
                poses.append(mol)
    return poses

In [ ]:
# visualize the docking poses in an interactive manner, browsing docked poses using an embedded slider
def update_viewer(pose_index):

    view = py3Dmol.view(width=1200, height=900)

    # Add the protein model
    view.addModel(protein_pdb, 'pdb')
    view.setStyle({'model': 0}, {'cartoon': {'color': 'white', 'opacity': 0.7}})
    view.setViewStyle({'style':'outline','color':'black','width':0.03})
    Prot=view.getModel()
    Prot.setStyle({'cartoon':{'arrows':True, 'tubes':True, 'style':'oval', 'color':'white'}})
    view.addSurface(py3Dmol.VDW,{'opacity':0.4,'color':'white'})

    # Add the selected docking pose
    pose = poses[pose_index]
    pose_block = Chem.MolToMolBlock(pose)
    # color = "#"+''.join([random.choice('0123456789ABCDEF') for _ in range(6)])
    view.addModel(pose_block, 'mol')
    view.setStyle({'model': 1}, {'stick': {'radius': 0.3, 'colorscheme': 'magentaCarbon'}})
    view.addSurface(py3Dmol.VDW, {'opacity': 0.7, 'colorscheme': 'magentaCarbon'}, {'model': 1})
    score = round(confidence_scores[pose_index], 3)
    score_color = "green" if score > 0 else "blue" if score >= -1.5 else "red"
    print(f"Loaded {ansi_color(ligand_name, 'magenta')} with confidence score: {ansi_color(confidence_scores[pose_index], score_color)}")
    view.zoomTo()
    return view.update()


In [ ]:
# Load the protein model
with open(protein_file_path, 'r') as f:
    protein_pdb = f.read()

# Specify the directory containing the dock poses in SDF format for a specific ligand
poses = load_poses_from_sdf(docking_ligand_dir)

# Verify the number of poses loaded
print(f"Number of poses loaded: {len(poses)}")

In [ ]:
# Load confidence scores from output.json
output_json_path = os.path.join(docking_ligand_dir, 'output.json')
with open(output_json_path, 'r') as file:
    data = json.load(file)
    confidence_scores = data['position_confidence']  # list of floats
print(confidence_scores)

In [ ]:
# Create a slider widget
pose_slider = widgets.IntSlider(
    value=0,
    min=0,
    max=len(poses) - 1,
    step=1,
    description='Pose:',
    continuous_update=False,
    orientation='horizontal',
    readout=True,
    readout_format='d'
)

# Link the slider to the viewer update function
widgets.interact(update_viewer, pose_index=pose_slider)